# SearchExecutor GPT-5-mini Smoke Test

Use this notebook to exercise `SearchExecutor` with OpenAI's `gpt-5-mini` model and the Firecrawl MCP server. Make sure the kernel sees `OPENAI_API_KEY` and that `npx firecrawl-mcp` works locally before running.

Steps:
1. Confirm Node.js 18+ and `npx` can launch `firecrawl-mcp`.
2. Export your OpenAI API key into this environment (e.g., `%env OPENAI_API_KEY=...`).
3. Run the cells in order to initialize the agent, stream updates, and invoke the Autogen-compatible `on_messages` helper.


In [5]:
import os
from langchain_openai import ChatOpenAI
from search_agent.execution import SearchExecutor

MODEL_NAME = "gpt-5-mini"
TARGET_QUERY = "What is a2a introduced by google"

if "OPENAI_API_KEY" not in os.environ:
    raise RuntimeError("Set OPENAI_API_KEY in this kernel before proceeding.")

# Note: gpt-5-mini only supports temperature=1.0 (default)
chat_model = ChatOpenAI(
    model=MODEL_NAME,
    api_key=os.getenv('OPENAI_API_KEY'),
    base_url=os.getenv('OPENAI_BASE_URL')
)

search_agent = SearchExecutor(model=chat_model)

# testing search Agent

In [6]:
# Test SearchExecutor with pure LangChain interface
search_agent = SearchExecutor(model=chat_model)
await search_agent.start()
print("SearchExecutor started.")


SearchExecutor started.


In [7]:
async def stream_query(agent, query: str):
    updates = []
    async for chunk in agent.stream(query):
        print(chunk)
        updates.append(chunk)
    return updates

stream_updates = await stream_query(search_agent, TARGET_QUERY)


{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 123, 'prompt_tokens': 2119, 'total_tokens': 2242, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D1E53J2zJrusRbmpy2EMWFOeWasjH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--018c717c-e502-4955-a3dc-5e58729afc10-0', tool_calls=[{'name': 'firecrawl_search', 'args': {'query': "Google 'A2A' introduced by Google what is A2A Google 'A2A' announcement", 'limit': 10, 'sources': [{'type': 'web'}]}, 'id': 'call_2XDW9tSMLgsSl89UiMtzwp2b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2119, 'output_tokens': 123, 'total_tokens': 2242, 'input_to

In [8]:
await search_agent.close()
print("SearchExecutor closed.")


SearchExecutor closed.
